In [ ]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, BasicAer
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
"""
    a in the register of n qubits to which we have to add the constant
    c is the constant
    g is the register of n-1 dirty ancilla qubits
    m is the qubit where we will save the carry
"""

def carry_gate(circuit,a,g,m,c,n):
    circuit.cx(g[n-2],m)
    circuit.barrier(a,g,m)
    for i in range(n-1,1,-1):
        if(c[i]=='1'):
            circuit.cx(a[i],g[i-1])
            circuit.x(a[i])
        circuit.ccx(g[i-2],a[i],g[i-1])
    circuit.barrier(a,g,m)
    if(c[1]=='1'):
        circuit.cx(a[1],g[0])
        circuit.x(a[1])
    if(c[0]=='1'):
        circuit.ccx(a[0],a[1],g[0])
    circuit.barrier(a,g,m)
    for i in range(2,n):
        circuit.ccx(g[i-2],a[i],g[i-1])
    circuit.barrier(a,g,m)
    circuit.cx(g[n-2],m)
    circuit.barrier(a,g,m)
    for i in range(n-1,1,-1):
        circuit.ccx(g[i-2],a[i],g[i-1])
    circuit.barrier(a,g,m)
    if(c[0]=='1'):
        circuit.ccx(a[0],a[1],g[0])
    if(c[1]=='1'):
        circuit.x(a[1])
        circuit.cx(a[1],g[0])
    circuit.barrier(a,g,m)
    for i in range(2,n):
        circuit.ccx(g[i-2],a[i],g[i-1])
        if(c[i]=='1'):
            circuit.x(a[i])
            circuit.cx(a[i],g[i-1])
    circuit.barrier(a,g,m)

In [ ]:
def test(A,c,n):
    g1=0
    x = QuantumRegister(n)
    g = QuantumRegister(n-1)
    m = QuantumRegister(1)
    clas_A = ClassicalRegister(n)
    clas_g = ClassicalRegister(n-1)
    clas_m = ClassicalRegister(1)
    circuit = QuantumCircuit(x,g,m,clas_A,clas_g,clas_m)
    s_c=bin(int(c))[2:].zfill(n)
    s_c=s_c[::-1]
    s_A=bin(int(A))[2:].zfill(n)
    for i in range(0, n):
        if s_A[i] == '1':
            circuit.x(x[n-1-i])
    carry_gate(circuit,x,g,m,s_c,n)
    for i in range(0,n):
        circuit.measure(x[i],clas_A[i])
    for i in range(0,n-1):
        circuit.measure(g[i],clas_g[i])
    circuit.measure(m,clas_m)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_A = output[2]
    output_g = output[1]
    output_m = output[0]
    res_A = int(output_A, 2)
    res_g = int(output_g, 2)
    res_m = int(output_m, 2)
    print(A,res_A,c,(A+c)//(2**n),res_g,res_m)
    if(res_m!=(A+c)//(2**n) or res_A!=A or res_g!=g1):
        print('found!')

In [ ]:
for i in range (0,2**4):
    for j in range (0,2**4):
        test(i,j,4)
        print('-----------')